## Wiki Price Download via API

#### Author: Ryan Gan

#### Date Created:  2018-06-10

Downloading stock ticker closing price data for the last month from the Wiki price dataset on Quandl site. I am going to use the python requests pacakge.

In [1]:
# import requests package
import requests
# http authentication
from requests.auth import HTTPBasicAuth
# import json
import simplejson as json
# import pandas
import pandas as pd
# import pandas io json to normalize json files
from pandas.io.json import json_normalize
# import config for api_key
import config

### Setting up API request for Wiki json file

Setting up API request for WIKI prices from May 1<sup>st</sup> 2018 to June 10<sup>th</sup> 2018.

In [11]:
url_api = ('https://www.quandl.com/api/v3/datatables/WIKI/PRICES.json?' +
           '&date.gte=2017-05-01&date.lte=2017-05-31&api_key=' + config.api_key)

Sending request for Wiki price data. I'm looking for outcome status code of 200, which means download was okay.

In [12]:
# send request
r = requests.get(url_api)
# print status
print(r.status_code)

200


View file type.

In [13]:
r.headers['content-type']

'application/json; charset=utf-8'

Parse the url request to a jason file.

In [14]:
# parse url request to json
j = r.json()
# pretty print json; suppressed print because it's a lot of lines
#print(json.dumps(j, indent=2, sort_keys=True))

### Converting json to pandas dataframe

To convert json file to pandas dataframe I need to get a couple things out of the nested json file. First I'm going to pull out the string list of variables using the 'json_normalize' function from the pandas package.

In [15]:
# use json normalize to access the datatable, and columns, and output a vector of the variable names
col_names = json_normalize(j['datatable'], 'columns')['name']
# print col_names
print(col_names)

0          ticker
1            date
2            open
3            high
4             low
5           close
6          volume
7     ex-dividend
8     split_ratio
9        adj_open
10       adj_high
11        adj_low
12      adj_close
13     adj_volume
Name: name, dtype: object


Now that I have a list of column names, I'm going to extract the data values from the table using the same 'json_normalize' function.

In [18]:
# extract data values from datatable
df = json_normalize(j['datatable'], 'data')
# add column names
df.columns = col_names
# view first couple rows
df.head()

name,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,2017-05-01,55.50,55.9900,55.36,55.80,1897018.0,0.0,1.0,55.263235,55.751145,55.123833,55.561956,1897018.0
1,A,2017-05-02,55.85,55.9700,55.51,55.93,1673735.0,0.0,1.0,55.611742,55.731230,55.273193,55.691401,1673735.0
2,A,2017-05-03,55.68,56.2575,55.61,56.16,1702565.0,0.0,1.0,55.442468,56.017504,55.372766,55.920420,1702565.0
3,A,2017-05-04,56.20,56.7500,56.16,56.70,2609703.0,0.0,1.0,55.960249,56.507903,55.920420,56.458116,2609703.0
4,A,2017-05-05,56.85,56.9600,56.59,56.64,1567046.0,0.0,1.0,56.607476,56.717007,56.348585,56.398372,1567046.0


View tail.

In [19]:
df.tail()

name,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
9995,CHGG,2017-05-12,11.62,11.71,11.390,11.53,590292.0,0.0,1.0,11.62,11.71,11.390,11.53,590292.0
9996,CHGG,2017-05-15,11.55,11.68,11.450,11.66,925798.0,0.0,1.0,11.55,11.68,11.450,11.66,925798.0
9997,CHGG,2017-05-16,11.45,11.68,11.250,11.65,1000613.0,0.0,1.0,11.45,11.68,11.250,11.65,1000613.0
9998,CHGG,2017-05-17,11.71,11.71,11.440,11.49,870489.0,0.0,1.0,11.71,11.71,11.440,11.49,870489.0
9999,CHGG,2017-05-18,11.41,11.55,11.265,11.54,780455.0,0.0,1.0,11.41,11.55,11.265,11.54,780455.0


Saving CSV file for bokeh plot and heroku app.

In [21]:
df.to_csv('./data/wiki_stock_price.csv', sep=',')